In [3]:
pip install psycopg2

  Using cached psycopg2-2.9.9-cp312-cp312-macosx_14_0_arm64.whl
Note: you may need to restart the kernel to use updated packages.


# Indigo Flight Data in Database

In [6]:
import http.client
import json
import psycopg2
from datetime import datetime

# Step 1: Fetch Data from the API
conn = http.client.HTTPSConnection("tripadvisor16.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "939577bcd4msh51c844c189119f7p129984jsnf3e058bd1f56",  # Replace with your actual API key
    'x-rapidapi-host': "tripadvisor16.p.rapidapi.com"
}

# Make API request
conn.request("GET", "/api/v1/flights/searchFlights?sourceAirportCode=BOM&destinationAirportCode=DEL&date=2024-10-17&itineraryType=ONE_WAY&sortOrder=ML_BEST_VALUE&numAdults=1&numSeniors=0&classOfService=ECONOMY&pageNumber=1&nearby=yes&nonstop=yes&currencyCode=INR&airlines=6E", headers=headers)

res = conn.getresponse()
data = res.read()
decoded_data = data.decode("utf-8")

# Parse JSON response
response = json.loads(decoded_data)

# Step 2: Connect to PostgreSQL
conn = psycopg2.connect("dbname=TransitGuide user=postgres password=0000")
cur = conn.cursor()

# SQL Insert Query
insert_query = """
    INSERT INTO IndigoFlightsTable (source, destination, flight_number, departure_date, departure_time, arrival_time, fare, class)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
"""

# Step 3: Parse and Insert Data
flights = response.get('data', {}).get('flights', [])

for flight in flights:
    for segment in flight['segments']:
        for leg in segment['legs']:
            source = leg['originStationCode']
            destination = leg['destinationStationCode']
            flight_number = leg['flightNumber']
            
            # Convert departure and arrival to separate date and time
            departure_datetime = datetime.fromisoformat(leg['departureDateTime'])
            arrival_datetime = datetime.fromisoformat(leg['arrivalDateTime'])
            departure_date = departure_datetime.date()
            departure_time = departure_datetime.time()
            arrival_time = arrival_datetime.time()
            
            # Get fare
            fare = flight['purchaseLinks'][0]['totalPricePerPassenger']  # Get fare from the first purchase link
            class_of_service = leg['classOfService']
            
            # Insert into database
            cur.execute(insert_query, (
                source, destination, flight_number, 
                departure_date, departure_time, 
                arrival_time, fare, class_of_service
            ))

# Commit the transaction
conn.commit()

# Close connection
cur.close()
conn.close()

print("Data inserted successfully.")

Data inserted successfully.


In [7]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 21.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 17.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import psycopg2
import pandas as pd

# Load the CSV file into a pandas DataFrame
csv_file_path = '/Users/mbhanu/Downloads/airport_data.csv'
df = pd.read_csv(csv_file_path)

# Connect to PostgreSQL
conn = psycopg2.connect("dbname=TransitGuide user=postgres password=0000")
cur = conn.cursor()


# Insert the data into the AirportData table
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO AirportData (type, name, latitude_deg, longitude_deg, city, airport_code)
    VALUES (%s, %s, %s, %s, %s, %s);
    """
    cur.execute(insert_query, (row['type'], row['name'], row['latitude_deg'], row['longitude_deg'], row['city'], row['airport_code']))

# Commit the transaction
conn.commit()

# Close connection
cur.close()
conn.close()

print("Data inserted successfully.")

Data inserted successfully.


In [9]:
pip install flask

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from rapidfuzz import process, fuzz

# Load the CSV data into a DataFrame and preprocess city names
airport_df = pd.read_csv('assets/airport_data.csv', usecols=['city', 'airport_code'])

# Preprocess city names to be case-insensitive and formatted as Title Case
airport_df['city'] = airport_df['city'].str.strip().str.title()


def get_closest_city(user_city):
    """
    Find the closest matching city to the user's input using fuzzy matching.
    """
    try:
        # List of available city names
        city_list = airport_df['city'].tolist()

        # Perform fuzzy matching
        closest_match, score, _ = process.extractOne(
            user_city.strip().title(), city_list, scorer=fuzz.ratio
        )

        # Debugging: Print matching details
        print(f"Input: {user_city}, Closest Match: {closest_match}, Score: {score}")

        # Threshold to ensure it's a valid match
        if score >= 70:  # 70 is a lenient similarity threshold
            return closest_match
        else:
            return None
    except Exception as e:
        print(f"Error finding closest city: {e}")
        return None


def get_airport_code(user_city):
    """
    Get the airport code for the given city.
    """
    try:
        # Find the closest matching city
        closest_city = get_closest_city(user_city)

        if closest_city:
            # Filter the DataFrame to find the airport code for the closest city
            result = airport_df[airport_df['city'] == closest_city]['airport_code']
            return result.iloc[0] if not result.empty else None
        else:
            return None
    except Exception as e:
        print(f"Error fetching airport code for {user_city}: {e}")
        return None


# Main script to accept user input
if __name__ == "__main__":
    while True:
        user_city = input("Enter the name of a city (or type 'exit' to quit): ").strip()
        if user_city.lower() == 'exit':
            print("Exiting the program. Goodbye!")
            break

        airport_code = get_airport_code(user_city)
        if airport_code:
            print(f"The airport code for the closest match to '{user_city}' is: {airport_code}")
        else:
            print(f"Could not find a valid match for '{user_city}'. Please try again.")
            

Input: Dilli, Closest Match: Diu, Score: 50.0
Could not find a valid match for 'Dilli'. Please try again.
Input: Bombay, Closest Match: Coimbatore, Score: 50.0
Could not find a valid match for 'Bombay'. Please try again.
Input: Delli, Closest Match: New Delhi, Score: 57.14285714285714
Could not find a valid match for 'Delli'. Please try again.
Input: vishakaptnam, Closest Match: Vishakhapatnam, Score: 92.3076923076923
The airport code for the closest match to 'vishakaptnam' is: VTZ
Input: visakapatnam, Closest Match: Vishakhapatnam, Score: 92.3076923076923
The airport code for the closest match to 'visakapatnam' is: VTZ
Input: ViSakaPatnam, Closest Match: Vishakhapatnam, Score: 92.3076923076923
The airport code for the closest match to 'ViSakaPatnam' is: VTZ
Input: MUmbaI, Closest Match: Mumbai, Score: 100.0
The airport code for the closest match to 'MUmbaI' is: BOM
Input: biombay, Closest Match: Coimbatore, Score: 47.05882352941176
Could not find a valid match for 'biombay'. Please tr

In [ ]:
from config import get_globalview_db_connection

def get_flights(source, destination, journey_date):
    try:
        conn = get_globalview_db_connection()
        cursor = conn.cursor()
        query = """
            SELECT flight_number, source_city, destination_city, departure_date, arrival_time, fare, airline
            FROM global_flights
            WHERE source_city = %s AND destination_city = %s AND DATE(departure_date) = %s
            ORDER BY fare ASC
            LIMIT 50;
        """
        cursor.execute(query, (source, destination, journey_date))
        results = cursor.fetchall()
        conn.close()

        flights = []
        for row in results:
            flights.append({
                "flight_number": row[0],
                "source_city": row[1],
                "destination_city": row[2],
                "departure_date": row[3],
                "arrival_time": row[4],
                "fare": row[5],
                "airline": row[6],
            })
        return flights
    except Exception as e:
        print(f"Error querying global flights: {e}")
        return []
    
get_flights('BOM','DEL','2024-11-29')


Error querying global flights: tuple index out of range


[]